<a href="https://colab.research.google.com/github/koseemre/ITU-Coding-and-Projects/blob/master/ML/AAPL_ann_indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np
import pandas as pd

from IPython.display import display
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (18, 12)

import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
!pip install quandl
import quandl
import sys


Using TensorFlow backend.


    100% |████████████████████████████████| 61kB 3.7MB/s 
    100% |████████████████████████████████| 51kB 8.1MB/s 
    100% |████████████████████████████████| 2.2MB 8.0MB/s 
    100% |████████████████████████████████| 102kB 18.4MB/s 
    100% |████████████████████████████████| 430kB 14.7MB/s 
    100% |████████████████████████████████| 256kB 12.8MB/s 
  Running setup.py bdist_wheel for inflection ... - done
  Stored in directory: /content/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
  Running setup.py bdist_wheel for pycparser ... - \ done
  Stored in directory: /content/.cache/pip/wheels/c0/a1/27/5ba234bd77ea5a290cbf6d675259ec52293193467a12ef1f46
Successfully built inflection pycparser


In [0]:

def plot_ann_prediction(actual, prediction, title='AMAZON Real vs Prediction', y_label='Price USD', x_label='Trading Days'):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    # Add labels
    plt.ylabel(y_label)
    plt.xlabel(x_label)

    # Plot actual and predicted close values

    plt.plot(actual, '#00FF00', label='Adjusted Close')
    plt.plot(prediction, '#0000FF', label='Predicted Close')

    # Set title
    ax.set_title(title)
    ax.legend(loc='upper left')


    plt.show()

In [0]:
quandl.ApiConfig.api_key = 't7wV_HsxHcR6HfpViu-K'

In [0]:
def get_stock_data(stock_name, normalize=True, ma=[]):
    
    df = quandl.get_table('WIKI/PRICES', ticker = stock_name)
    df.drop(['ticker', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'ex-dividend', 'adj_volume', 'split_ratio'], 1, inplace=True)
    df.set_index('date', inplace=True)
    
    # Renaming all the columns so that we can use the old version code
    #df.rename(columns={'adj_open': 'Open', 'adj_high': 'High', 'adj_low': 'Low', 'adj_volume': 'Volume', 'adj_close': 'Adj Close'}, inplace=True)
    
           
    # Move Adj Close to the rightmost for the ease of training
    adj_close = df['close']
    df.drop(labels=['close'], axis=1, inplace=True)
    df = pd.concat([df, adj_close], axis=1)
      
    return df

In [0]:
ds = get_stock_data('AAPL')

print(ds)

               open     high       low      volume    close
date                                                       
1980-12-12   28.750   28.870   28.7500   2093900.0   28.750
1980-12-15   27.380   27.380   27.2500    785200.0   27.250
1980-12-16   25.370   25.370   25.2500    472000.0   25.250
1980-12-17   25.870   26.000   25.8700    385900.0   25.870
1980-12-18   26.630   26.750   26.6300    327900.0   26.630
1980-12-19   28.250   28.380   28.2500    217100.0   28.250
1980-12-22   29.630   29.750   29.6300    166800.0   29.630
1980-12-23   30.880   31.000   30.8800    209600.0   30.880
1980-12-24   32.500   32.630   32.5000    214300.0   32.500
1980-12-26   35.500   35.620   35.5000    248100.0   35.500
1980-12-29   36.000   36.130   36.0000    415900.0   36.000
1980-12-30   35.250   35.250   35.1200    307500.0   35.120
1980-12-31   34.250   34.250   34.1300    159600.0   34.130
1981-01-02   34.500   34.750   34.5000     96700.0   34.500
1981-01-05   33.870   33.870   33.7500  

In [0]:
#ds = get_stock_data('AAPL', ma=[50, 100, 200])
ds = get_stock_data('AAPL')
dataset = ds.iloc[:, [0,1,2,3,4]].values   #open high low volume close


In [0]:
w, h = 1, ds.shape[0];
onBalanceVolume = [[0 for x in range(w)] for y in range(h)]
onBalanceVolume[0][0]=0

for i in range(1, dataset.shape[0]):
  if dataset[i][4] > dataset[i-1][4]:
    onBalanceVolume[i][0] = onBalanceVolume[i-1][0] + dataset[i][3]
  elif dataset[i][4] <  dataset[i-1][4]:  
    onBalanceVolume[i][0] = onBalanceVolume[i-1][0] - dataset[i][3]
  else:
    onBalanceVolume[i][0] = onBalanceVolume[i-1][0]
    

o_b_v_array = np.asarray(onBalanceVolume)

In [0]:
w, h = 1, ds.shape[0]-10;
William_R = [[0 for x in range(w)] for y in range(h)]


for i in range(10, dataset.shape[0]):
  highest = 0.0
  lowest = sys.float_info.max 
  closed = dataset[i][4]
  for counter in range(0, 10):
      if dataset[i-counter][1] > highest:
        highest = dataset[i-counter][1]
      if dataset[i-counter][2] < lowest:
        lowest = dataset[i-counter][2] 
  w_r =  (highest-closed)*-100/(highest-lowest)
  William_R[i-10][0] = w_r

  
w_r_array = np.asarray(William_R)

In [0]:
def movingAverage(values,window):
    weigths = np.repeat(1.0, window)/window
    smas = np.convolve(values, weigths, 'valid')
    return smas # as a numpy array
  
  
#cci implementation
typPrices=[]
MDar = []
cci = []


tf = 14
sma = 20
i  = 0
while i < dataset.shape[0]:
  tp = (dataset[i][1] + dataset[i][2] + dataset[i][4])/3
  typPrices.append(tp)
  i = i+1
  
SMATP = movingAverage(typPrices,sma)
typPrices =  typPrices[sma-1:] 


y = tf
while y < len(SMATP):
  considerationTP = typPrices[y-tf:y]
  considerationSMATP= SMATP[y-tf:y]
  MDs = 0
  z=0
  while z < len(considerationTP):
        curMD = abs(considerationTP[z] - considerationSMATP[z])
        MDs += curMD
        z += 1
  MD = MDs/tf
  MDar.append(MDs)
  
  y+=1  



typPrices = typPrices [14:]
SMATP =SMATP [14:]
t = 0
while t < len(SMATP):
    ccis = (typPrices[t]-SMATP[t]) / (0.015*MDar[t])
    cci.append(ccis)
    t+=1
  
  
  
cci_array = np.asarray(cci)


In [0]:
#rate of change
w, h = 1, ds.shape[0]-10;
Rate_of_Change = [[0 for x in range(w)] for y in range(h)]

for i in range(10, dataset.shape[0]):
  roc =  (dataset[i][4]-dataset[i-5][4])
  Rate_of_Change[i-10][0] = roc
 

roc_array = np.asarray(Rate_of_Change)


In [0]:
dataset_final = ds.iloc[10:ds.shape[0], [0,3,4]].values
dataset_final = np.c_[ dataset_final, w_r_array] 

print(dataset_final.shape)

# Feature Scaling
scaler  = MinMaxScaler(feature_range=(0, 1))
dataset_scaled = scaler.fit_transform(dataset_final)
X = dataset_scaled[:, [0,1,3]]
y = dataset_scaled[:, 2]



(9390, 4)


In [0]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, shuffle=False)
# Sizes of train_ds, test_ds
dataset_sz = X.shape[0]
train_sz = X_train.shape[0]
test_sz = X_test.shape[0]

############ Building the ANN ############
# Initialising the ANN
regressor = Sequential()

# Adding the input layer and the first hidden layer and Drop out Regularization
regressor.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu', input_dim = 3))
regressor.add(Dropout(.2))

# Adding the first hidden layer and Drop out Regularization
regressor.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dropout(.2))

# Adding the second hidden layer and Drop out Regularization
regressor.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dropout(.2))

# Adding the third hidden layer and Drop out Regularization
regressor.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dropout(.2))

# Adding the output layer
regressor.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'linear'))

# Compiling the ANN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the ANN to the Training set
regressor.fit(X_train, y_train, batch_size = 32, epochs = 20)




# Save Trained Model
regressor.save('TICKER-BKP.h5')

# deletes the existing model
del regressor

# load Trained Model
regressor = load_model('TICKER-BKP.h5')




Epoch 1/20
8451/8451 [==============================] - 3s 389us/step - loss: 0.0033
Epoch 2/20
8451/8451 [==============================] - 2s 251us/step - loss: 8.5112e-04
Epoch 3/20
8451/8451 [==============================] - 2s 252us/step - loss: 9.9121e-04
Epoch 4/20
8451/8451 [==============================] - 2s 252us/step - loss: 8.7482e-04
Epoch 5/20
8451/8451 [==============================] - 2s 248us/step - loss: 7.0990e-04
Epoch 6/20
8451/8451 [==============================] - 2s 251us/step - loss: 6.7435e-04
Epoch 7/20
8451/8451 [==============================] - 2s 258us/step - loss: 6.1320e-04
Epoch 8/20
 480/8451 [>.............................] - ETA: 1s - loss: 5.1222e-04

8451/8451 [==============================] - 2s 253us/step - loss: 4.6627e-04
Epoch 9/20
8451/8451 [==============================] - 2s 254us/step - loss: 4.9654e-04
Epoch 10/20
8451/8451 [==============================] - 2s 249us/step - loss: 4.7443e-04
Epoch 11/20
8451/8451 [==============================] - 2s 253us/step - loss: 5.0730e-04
Epoch 12/20
8451/8451 [==============================] - 2s 251us/step - loss: 3.8747e-04
Epoch 13/20
8451/8451 [==============================] - 2s 249us/step - loss: 4.3390e-04
Epoch 14/20
8451/8451 [==============================] - 2s 253us/step - loss: 4.7123e-04
Epoch 15/20
 704/8451 [=>............................] - ETA: 1s - loss: 2.5518e-04

8451/8451 [==============================] - 2s 247us/step - loss: 3.9120e-04
Epoch 16/20
8451/8451 [==============================] - 2s 246us/step - loss: 3.7707e-04
Epoch 17/20
8451/8451 [==============================] - 2s 251us/step - loss: 3.4531e-04
Epoch 18/20
8451/8451 [==============================] - 2s 252us/step - loss: 3.9061e-04
Epoch 19/20
8451/8451 [==============================] - 2s 250us/step - loss: 4.3116e-04
Epoch 20/20
8451/8451 [==============================] - 2s 252us/step - loss: 4.1221e-04


In [0]:
############ Predict & Test the Model ############
y_test_temp = dataset_final[X_train.shape[0]:ds.shape[0]-1,2]
############ Predict & Test the Model ############
inputs =  np.array(X_test)
predicted_stock_price = regressor.predict(inputs)
predicted_decimal = []
#print(y_test_temp.shape)
#print(predicted_stock_price.shape)
for i in range(y_test_temp.shape[0]):
  predicted_decimal.append((y_test_temp[i] * predicted_stock_price[i])/y_test[i])
                            
                            
predicted_decimal = np.array(predicted_decimal)
#print(y_test_temp)
#print(predicted_decimal)

In [0]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='koseemr', api_key='E7jrNXXlB05yFJkjP3iU')
trace0 = go.Scatter(
    x = np.arange(predicted_decimal.shape[0]),
    y = predicted_decimal,
    mode = 'lines',
    name = 'predicted'
)
trace1 = go.Scatter(
    x = np.arange(y_test_temp.shape[0]),
    y = y_test_temp,
    mode = 'lines',
    name = 'real'
)
data = [trace0,trace1]
layout = go.Layout(
    title='Predictions for Apple-ANN',
    xaxis=dict(
        title='days',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='price of stock',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='line-mode')


High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~koseemr/0 or inside your plot.ly account where it is named 'line-mode'


In [0]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test_temp, predicted_decimal)
print(mse)
mse = 0.0
for i in range(y_test_temp.shape[0]):
  mse = mse + (y_test_temp[i]-predicted_decimal[i])*(y_test_temp[i]-predicted_decimal[i])
  
mse = mse / y_test_temp.shape[0]
print(mse)


29.259506490995424
[29.259502]
